In [1]:
from VariationalBayes import ScalarParam, ModelParamsDict, VectorParam, PosDefMatrixParam
from VariationalBayes.NormalParams import MVNParam, UVNParam, UVNParamVector
from VariationalBayes.GammaParams import GammaParam
from VariationalBayes.ExponentialFamilies import \
    UnivariateNormalEntropy, MultivariateNormalEntropy, GammaEntropy, \
    MVNPrior, UVNPrior, GammaPrior


from autograd import grad, hessian, jacobian, hessian_vector_product
import autograd.numpy as np
import autograd.numpy.random as npr
import autograd.scipy as asp
import scipy as sp

import copy
from scipy import optimize

In [2]:
# Load data saved by stan_results_to_json.R and run_stan.R in LRVBLogitGLMM.
import os
import json

simulate_data = False
prior_par = ModelParamsDict('Prior Parameters')

if not simulate_data:
    analysis_name = 'simulated_data_small'
    #analysis_name = 'simulated_data_large'

    data_dir = os.path.join(os.environ['GIT_REPO_LOC'], 'LRVBLogitGLMM/LogitGLMMLRVB/inst/data/')
    json_filename = os.path.join(data_dir, '%s_stan_dat.json' % analysis_name)
    json_output_filename = os.path.join(data_dir, '%s_python_vb_results.json' % analysis_name)

    json_file = open(json_filename, 'r')
    json_dat = json.load(json_file)
    json_file.close()

    stan_dat = json_dat['stan_dat']
    vp_base = json_dat['vp_base']

    print stan_dat.keys()
    K = stan_dat['K'][0]
    NObs = stan_dat['N'][0]
    NG = stan_dat['NG'][0]
    N = NObs / NG
    y_g_vec = np.array(stan_dat['y_group'])
    y_vec = np.array(stan_dat['y'])
    x_mat = np.array(stan_dat['x'])
    
    # Define a class to contain prior parameters.
    prior_par.push_param(VectorParam('beta_prior_mean', K, val=np.array(stan_dat['beta_prior_mean'])))
    prior_par.push_param(PosDefMatrixParam('beta_prior_var', K, val=np.array(stan_dat['beta_prior_var'])))

    prior_par.push_param(ScalarParam('mu_prior_mean', val=stan_dat['mu_prior_mean'][0]))
    prior_par.push_param(ScalarParam('mu_prior_var', val=stan_dat['mu_prior_var'][0]))

    prior_par.push_param(ScalarParam('tau_prior_alpha', val=stan_dat['tau_prior_alpha'][0]))
    prior_par.push_param(ScalarParam('tau_prior_beta', val=stan_dat['tau_prior_beta'][0]))

    # An index set to make sure jacobians match the order expected by R.
    prior_par_indices = copy.deepcopy(prior_par)
    prior_par_indices.set_name('Prior Indices')
    prior_par_indices.set_vector(np.array(range(prior_par_indices.vector_size())))
else:
    # Simulate data instead of loading it if you like
    N = 200     # observations per group
    K = 5      # dimension of regressors
    NG = 200      # number of groups

    # Generate data
    def Logistic(u):
        return np.exp(u) / (1 + np.exp(u))

    NObs = NG * N
    true_beta = np.array(range(5))
    true_beta = true_beta - np.mean(true_beta)
    true_mu = 0.
    true_tau = 40.0
    true_u = np.random.normal(true_mu, 1 / np.sqrt(true_tau), NG)

    x_mat = np.random.random(K * NObs).reshape(NObs, K) - 0.5
    y_g_vec = [ g for g in range(NG) for n in range(N) ]
    true_rho = Logistic(np.matmul(x_mat, true_beta) + true_u[y_g_vec])
    y_vec = np.random.random(NObs) < true_rho
    
    prior_par.push_param(VectorParam('beta_prior_mean', K, val=np.zeros(K)))
    prior_par.push_param(PosDefMatrixParam('beta_prior_var', K, val=10 * np.eye(K)))

    prior_par.push_param(ScalarParam('mu_prior_mean', val=0))
    prior_par.push_param(ScalarParam('mu_prior_var', val=2))

    prior_par.push_param(ScalarParam('tau_prior_alpha', val=3.0))
    prior_par.push_param(ScalarParam('tau_prior_beta', val=10.0))

print N * NG
print np.mean(y_vec)

[u'y_group', u'mu_prior_var', u'mu_prior_t', u'mu_prior_var_c', u'K', u'beta_prior_var', u'tau_prior_beta', u'N', u'mu_prior_mean_c', u'mu_prior_epsilon', u'mu_prior_mean', u'y', u'x', u'NG', u'beta_prior_mean', u'tau_prior_alpha']
1000
0.324


In [25]:
print vp_base.keys()
print vp_base['u_info_min']

[u'mu_info', u'tau_beta', u'tau_beta_min', u'tau_alpha', u'beta_info', u'tau_alpha_min', u'beta_diag_min', u'n_groups', u'encoded_size', u'beta_loc', u'u_info_min', u'u', u'k_reg', u'mu_loc', u'mu_info_min']


NameError: name 'u_info_min' is not defined

In [9]:
print vp_base['mu_info_min'][0]

# Build an object to contain a variational approximation to a K-dimensional multivariate normal.
glmm_par = ModelParamsDict('GLMM Parameters')

glmm_par.push_param(UVNParam('mu', min_info=vp_base['mu_info_min'][0]))
glmm_par.push_param(GammaParam('tau', min_shape=vp_base['tau_alpha_min'][0], min_rate=vp_base['tau_beta_min'][0]))
glmm_par.push_param(MVNParam('beta', K, min_info=vp_base['beta_diag_min'][0]))
glmm_par.push_param(UVNParamVector('u', NG, min_info=vp_base['u_info_min'][0]))

glmm_par['mu'].mean.set(0.1)
glmm_par['mu'].info.set(1.0)

glmm_par['tau'].shape.set(2.1)
glmm_par['tau'].rate.set(2.1)

glmm_par['beta'].mean.set(np.full(K, 0.))
glmm_par['beta'].info.set(np.eye(K))

glmm_par['u'].mean.set(np.full(NG, 0.))
glmm_par['u'].info.set(np.full(NG, 1))

print glmm_par


0.3
GLMM Parameters:
	mu:
mu_mean: 0.1
mu_info: 1.0
	tau:
tau_shape: 2.1
tau_rate: 2.1
	beta:
beta_mean:
[ 0.  0.  0.  0.  0.]
beta_info:
[[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
	u:
u_mean:
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
u_info:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [10]:
# Define moment parameters

moment_par = ModelParamsDict('Moment Parameters')
moment_par.push_param(VectorParam('e_beta', K))
moment_par.push_param(PosDefMatrixParam('e_beta_outer', K))
moment_par.push_param(ScalarParam('e_mu'))
moment_par.push_param(ScalarParam('e_mu2'))
moment_par.push_param(ScalarParam('e_tau'))
moment_par.push_param(ScalarParam('e_log_tau'))
moment_par.push_param(VectorParam('e_u', NG))
moment_par.push_param(VectorParam('e_u2', NG))

def set_moments(glmm_par, moment_par):
    moment_par['e_beta'].set(glmm_par['beta'].e())
    moment_par['e_beta_outer'].set(glmm_par['beta'].e_outer())
    moment_par['e_mu'].set(glmm_par['mu'].e())
    moment_par['e_mu2'].set(glmm_par['mu'].e_outer())
    moment_par['e_tau'].set(glmm_par['tau'].e())
    moment_par['e_log_tau'].set(glmm_par['tau'].e_log())
    moment_par['e_u'].set(glmm_par['u'].e())
    moment_par['e_u2'].set((glmm_par['u'].e_outer()))
    
set_moments(glmm_par, moment_par)

# Moment indices.
moment_indices = copy.deepcopy(moment_par)
moment_indices.set_vector(1 + np.array(range(moment_indices.vector_size())))

In [11]:
def decode_combined_parameters(combined_free_par_vec, glmm_par, prior_par):
    assert glmm_par.free_size() + prior_par.vector_size() == len(combined_free_par_vec) 
    glmm_par.set_free(combined_free_par_vec[0:glmm_par.free_size()])
    prior_par.set_vector(combined_free_par_vec[glmm_par.free_size():])

    
def encode_combined_parameters(glmm_par, prior_par):
    combined_free_par_vec = np.full(glmm_par.free_size() + prior_par.vector_size(), float('nan'))
    combined_free_par_vec[0:glmm_par.free_size()] = glmm_par.get_free()
    combined_free_par_vec[glmm_par.free_size():] = prior_par.get_vector()
    return combined_free_par_vec


In [12]:
def ELogPrior(prior_par, glmm_par_elbo):
    e_beta = glmm_par_elbo['beta'].mean.get()
    info_beta = glmm_par_elbo['beta'].info.get()
    cov_beta = np.linalg.inv(info_beta)
    beta_prior_info = np.linalg.inv(prior_par['beta_prior_var'].get())
    beta_prior_mean = prior_par['beta_prior_mean'].get()
    e_log_p_beta = MVNPrior(beta_prior_mean, beta_prior_info, e_beta, cov_beta)
    
    e_mu = glmm_par_elbo['mu'].mean.get()
    info_mu = glmm_par_elbo['mu'].info.get()
    var_mu = 1 / info_mu
    e_log_p_mu = UVNPrior(prior_par['mu_prior_mean'].get(), 1 / prior_par['mu_prior_var'].get(), e_mu, var_mu) 

    e_tau = glmm_par_elbo['tau'].e()
    e_log_tau = glmm_par_elbo['tau'].e_log()
    tau_prior_shape = prior_par['tau_prior_alpha'].get()
    tau_prior_rate = prior_par['tau_prior_beta'].get()
    e_log_p_tau = GammaPrior(tau_prior_shape, tau_prior_rate, e_tau, e_log_tau)
    
    return  e_log_p_beta + e_log_p_mu + e_log_p_tau
           

def DataLogLikelihood(x_mat, y_vec, e_beta, cov_beta, e_u, var_u, std_draws):
    z_mean = e_u + np.matmul(x_mat, e_beta)
    z_sd = np.sqrt(var_u + np.einsum('nk,kj,nj->n', x_mat, cov_beta, x_mat))
    z = np.einsum('i,j->ij', z_sd, std_draws) + np.expand_dims(z_mean, 1)

    # The sum is over observations and draws, so dividing by the draws size
    # gives the sum of sample expectations over the draws.
    # p = exp(z) / (1 + exp(z))
    # log(1 - p) = log(1 / (1 + exp(z))) = -log(1 + exp(z))
    logit_term = -np.sum(np.log1p(np.exp(z))) / std_draws.size
    y_term = np.sum(y_vec * z_mean)
    return y_term + logit_term


def RandomEffectLogLikelihood(e_u, var_u, e_mu, var_mu, e_tau, e_log_tau):
    return -0.5 * e_tau * np.sum(((e_mu - e_u) ** 2) + var_mu + var_u) + 0.5 * e_log_tau * len(e_u)

    
def Elbo(y_vec, x_mat, y_g_vec, glmm_par_elbo, std_draws, prior_par):
    e_beta = glmm_par_elbo['beta'].mean.get()
    info_beta = glmm_par_elbo['beta'].info.get()
    cov_beta = np.linalg.inv(info_beta)
    
    e_u = glmm_par_elbo['u'].mean.get()
    info_u = glmm_par_elbo['u'].info.get()
    var_u = 1 / info_u
    
    e_mu = glmm_par_elbo['mu'].mean.get()
    info_mu = glmm_par_elbo['mu'].info.get()
    var_mu = 1 / info_mu
    
    e_tau = glmm_par_elbo['tau'].e()
    e_log_tau = glmm_par_elbo['tau'].e_log()
        
    ll = \
        DataLogLikelihood(x_mat, y_vec, e_beta, cov_beta,
                          e_u[y_g_vec], var_u[y_g_vec], std_draws) + \
        RandomEffectLogLikelihood(e_u, var_u, e_mu, var_mu, e_tau, e_log_tau)

    e_log_prior = ELogPrior(prior_par, glmm_par_elbo)
    
    tau_shape = glmm_par_elbo['tau'].shape.get()
    tau_rate = glmm_par_elbo['tau'].rate.get()
    entropy = \
        UnivariateNormalEntropy(info_mu) + \
        MultivariateNormalEntropy(info_beta) + \
        UnivariateNormalEntropy(info_u) + \
        GammaEntropy(tau_shape, tau_rate)

    return ll[0] + e_log_prior[0] + entropy


class KLWrapper():
    def __init__(self, glmm_par, moment_par, prior_par, x_mat, y_vec, y_g_vec, num_draws):
        self.__glmm_par_ad = copy.deepcopy(glmm_par)
        self.__moment_par = copy.deepcopy(moment_par)
        self.__prior_par_ad = copy.deepcopy(prior_par)
        self.x_mat = x_mat
        self.y_vec = y_vec
        self.y_g_vec = y_g_vec
        draw_spacing = 1 / float(num_draws + 1)
        target_quantiles = np.linspace(draw_spacing, 1 - draw_spacing, num_draws)
        self.std_draws = sp.stats.norm.ppf(target_quantiles)

    def Eval(self, free_par_vec, verbose=False):
        self.__glmm_par_ad.set_free(free_par_vec)
        kl = -Elbo(self.y_vec, self.x_mat, self.y_g_vec,
                   self.__glmm_par_ad, self.std_draws, self.__prior_par_ad)
        if verbose: print kl
            
        # TODO: this is returning an array when it should be a scalar.
        return kl
    
    def ExpectedLogPrior(self, combined_free_par_vec):
        # Encode the glmm parameters first and the prior second.
        decode_combined_parameters(combined_free_par_vec, self.__glmm_par_ad, self.__prior_par_ad)
        e_log_prior = ELogPrior(self.__prior_par_ad, self.__glmm_par_ad)
        return e_log_prior[0]
        
    # Return a posterior moment of interest as a function of
    # unconstrained parameters.  In this case it is a bit silly,
    # but in full generality posterior moments may be a complicated
    # function of moment parameters.
    def GetMoments(self, free_par_vec):
        self.__glmm_par_ad.set_free(free_par_vec)
        set_moments(self.__glmm_par_ad, self.__moment_par)
        return self.__moment_par.get_vector()


# TODO: get the log prior derivatives, too.

kl_wrapper = KLWrapper(glmm_par, moment_par, prior_par, x_mat, y_vec, y_g_vec, 10)
KLGrad = grad(kl_wrapper.Eval)
KLHess = hessian(kl_wrapper.Eval)
MomentJacobian = jacobian(kl_wrapper.GetMoments)
KLHessVecProd = hessian_vector_product(kl_wrapper.Eval)  
free_par_vec = glmm_par.get_free()
print kl_wrapper.Eval(free_par_vec)

combined_free_par_vec = encode_combined_parameters(glmm_par, prior_par)
PriorHess = hessian(kl_wrapper.ExpectedLogPrior)
kl_wrapper.ExpectedLogPrior(combined_free_par_vec)


917.484930118


-3.5432527520990904

In [13]:
import timeit

time_num = 10

print 'Function time:'
print timeit.timeit(lambda: kl_wrapper.Eval(free_par_vec), number=time_num) / time_num

print 'Grad time:'
print timeit.timeit(lambda: KLGrad(free_par_vec), number=time_num) / time_num

print 'Hessian vector product time:'
print timeit.timeit(lambda: KLHessVecProd(free_par_vec, free_par_vec + 1), number=time_num) / time_num

print 'Moment jacobian time:'
print timeit.timeit(lambda: MomentJacobian(free_par_vec), number=time_num) / time_num

time_num = 1
print 'Prior Hessian time:'
print timeit.timeit(lambda: PriorHess(combined_free_par_vec), number=time_num) / time_num

# so slow
# print 'Hessian time:'
# print timeit.timeit(lambda: KLHess(free_par_vec), number=time_num) / time_num


Function time:
0.00148417949677
Grad time:
0.00717601776123
Hessian vector product time:
0.0137457132339
Moment jacobian time:
0.339930701256
Prior Hessian time:
1.25546884537


In [14]:
import time

init_par_vec = free_par_vec

# Optimize.
vb_time = time.time()
print 'Running BFGS'
vb_opt_bfgs = optimize.minimize(
    lambda par: kl_wrapper.Eval(par, verbose=True), init_par_vec,
    method='bfgs', jac=KLGrad, tol=1e-2, options={'maxiter': 5000, 'disp': True})

init_par_vec = free_par_vec
print 'Running Newton Trust Region'
vb_opt = optimize.minimize(
    lambda par: kl_wrapper.Eval(par, verbose=True),
    vb_opt_bfgs.x, method='trust-ncg', jac=KLGrad, hessp=KLHessVecProd,
    tol=1e-8, options={'maxiter': 5000, 'disp': True})

vb_time = time.time() - vb_time

glmm_par_opt = copy.deepcopy(glmm_par)
glmm_par_opt.set_free(vb_opt.x)
moment_jac = MomentJacobian(vb_opt.x)

print 'Done.'

print vb_time / 60

Running BFGS
917.489980118
573.748050203
1210.38249472
508.777179013
431.601965347
336.713383719
299.178221204
238.76290226
218.661302557
184.611999864
195.300228484
170.215139503
150.981197263
135.610930691
113.321530164
98.0523800522
87.2579695768
80.8103672595
76.5591051797
74.6197136489
73.6960236352
72.231798695
69.8151870091
73.1395409838
68.1604222002
69.4182174503
67.0497939729
65.6087236898
64.8327476475
64.6532373374
64.4431381686
64.3166870499
64.164979181
63.9916812521
63.8989211711
63.8317504319
63.808953448
63.7900766486
63.7699482082
63.7380975897
63.6858977592
63.5962274028
63.4357638849
63.2686801644
63.0908181472
62.8131704977
62.5462992414
62.4556003893
62.3772955962
62.335410936
62.3091527973
62.2909199917
62.2700147566
62.2486172582
62.2108744904
62.1434893059
62.0245521688
61.8327873079
61.9289367121
61.7146960858
61.5757177393
61.4172600974
61.3163151315
61.2278997381
61.131673126
61.0224888129
60.9754482593
60.9435159687
60.924916558
60.9067394644
60.8777740038


In [15]:
# print(glmm_par_opt)
if simulate_data:
    print true_beta
    print glmm_par_opt['beta']
    print '---------------\n'
    print true_tau
    print glmm_par_opt['tau'].e()

    e_u = glmm_par_opt['u'].e()
    info_u = glmm_par_opt['u'].info.get()
    var_u = 1 / info_u
    e_beta = glmm_par_opt['beta'].e()
    e_beta_outer = glmm_par_opt['beta'].e_outer()
    std_draws = kl_wrapper.std_draws

    rho_mean = e_u[y_g_vec] + np.matmul(x_mat, e_beta)
    rho_sd = np.sqrt(var_u[y_g_vec] + np.einsum('nk,kj,nj->n', x_mat, e_beta_outer, x_mat))
    z = np.einsum('i,j->ij', rho_sd, std_draws) + np.expand_dims(rho_mean, 1)
    logit_term = -np.einsum('ij->i', np.log1p(np.exp(z))) / std_draws.size

    print rho_sd
    print var_u[y_g_vec]
    # print np.mean(var_u)


In [16]:
# Check the random effect estimates.  This requires simulated data.
if simulate_data:
    from ggplot import *
    import pandas as pd
    %matplotlib inline
    
    print glmm_par_opt['mu'].e()
    print true_mu

    print glmm_par_opt['tau'].e()
    print true_tau

    plot_df = pd.DataFrame({ 'opt': glmm_par_opt['u'].mean.get(), 'true': true_u })
    print ggplot(plot_df, aes(x='true', y='opt')) + geom_point() + geom_abline(slope=1, intercept=0)
    
    plot_df = pd.DataFrame({ 'opt': glmm_par_opt['beta'].mean.get(), 'true': true_beta })
    print ggplot(plot_df, aes(x='true', y='opt')) + geom_point() + geom_abline(slope=1, intercept=0)
    
    plot_df = pd.DataFrame({ 'opt': logit_term, 'true': np.log(1 - true_rho) })
    print ggplot(plot_df, aes(x='true', y='opt')) + geom_point() + geom_abline(slope=1, intercept=0)

In [17]:
# LRVB with conjugate gradient.  This turns out to be way slower with any appreciable number of moments.
if False:
    from scipy.sparse.linalg import LinearOperator
    import sys

    # This will actually compute Hess^1 * moment_jac.T, leading to perhaps confusing
    # naming of "columns".  
    ObjHessVecProdLO = LinearOperator((vb_opt.x.size, vb_opt.x.size), lambda par: KLHessVecProd(vb_opt.x, par))
    # print moment_jac.T.shape
    # print ObjHessVecProdLO.shape
    # cg_res, info = scipy.sparse.linalg.cg(ObjHessVecProdLO, moment_jac.T)

    cg_time = time.time()
    lrvb_term = np.full(moment_jac.T.shape, float('nan'))
    for col in range(moment_jac.shape[0]):
        sys.stdout.write('.')
        sys.stdout.flush()
        cg_res, info = sp.sparse.linalg.cg(ObjHessVecProdLO, moment_jac[col, :])
        assert info == 0
        lrvb_term[:, col] = cg_res
    cg_time = time.time() - cg_time

    print 'all done dude'
else:
    cg_time = float('inf')

In [18]:
# Slow, but probably faster than using CG.
combined_free_par_vec = encode_combined_parameters(glmm_par_opt, prior_par)

hess_time = time.time()
kl_hess = KLHess(vb_opt.x)
log_prior_hess_full = PriorHess(combined_free_par_vec)
hess_time =  time.time() - hess_time
elbo_hess = -kl_hess

print 'hess_time: %f' % hess_time
print 'cg_time: %f' % cg_time

hess_time: 2.824652
cg_time: inf


In [19]:
glmm_inds = range(glmm_par_opt.free_size())
prior_inds = range(glmm_par_opt.free_size(), len(combined_free_par_vec))
log_prior_hess = log_prior_hess_full[np.ix_(prior_inds, glmm_inds)]

lrvb_cov = np.matmul(moment_jac, np.linalg.solve(kl_hess, moment_jac.T))

prior_indices = copy.deepcopy(prior_par)
prior_indices.set_vector(1 + np.array(range(prior_indices.vector_size())))

vp_indices = copy.deepcopy(glmm_par_opt)
vp_indices.set_vector(1 + np.array(range(vp_indices.vector_size())))

In [23]:
if not simulate_data:
    result_dict = { 'glmm_par_opt': glmm_par_opt.dictval(),
                    'vb_time': vb_time,'hess_time': hess_time, 
                    'moment_indices': moment_indices.dictval(),
                    'prior_indices': prior_indices.dictval(),
                    'vp_indices': vp_indices.dictval(),
                    'lrvb_cov': lrvb_cov.tolist(), 'moment_jac': moment_jac.tolist(),
                    'elbo_hess': elbo_hess.tolist(), 'log_prior_hess': log_prior_hess.tolist() }

    result_json = json.dumps(result_dict)
    json_file = open(json_output_filename, 'w')
    json_file.write(result_json)
    json_file.close()

    print(json_output_filename)

/home/rgiordan/Documents/git_repos/LRVBLogitGLMM/LogitGLMMLRVB/inst/data/simulated_data_small_python_vb_results.json


In [22]:
glmm_par_elbo = copy.deepcopy(glmm_par)

def DebugFun(glmm_par_elbo, prior_par):
    e_beta = glmm_par_elbo['beta'].mean.get()
    info_beta = glmm_par_elbo['beta'].info.get()
    cov_beta = np.linalg.inv(info_beta)

    e_u = glmm_par_elbo['u'].mean.get()
    info_u = glmm_par_elbo['u'].info.get()
    var_u = 1 / info_u

    e_mu = glmm_par_elbo['mu'].mean.get()
    info_mu = glmm_par_elbo['mu'].info.get()
    var_mu = 1 / info_mu

    e_tau = glmm_par_elbo['tau'].e()
    e_log_tau = glmm_par_elbo['tau'].e_log()
    
    tau_shape = glmm_par_elbo['tau'].shape.get()
    tau_rate = glmm_par_elbo['tau'].rate.get()

    e_tau = glmm_par_elbo['tau'].e()
    e_log_tau = glmm_par_elbo['tau'].e_log()
    tau_prior_shape = prior_par['tau_prior_alpha'].get()
    tau_prior_rate = prior_par['tau_prior_beta'].get()
    e_log_p_tau = GammaPrior(tau_prior_shape, tau_prior_rate, e_tau, e_log_tau)

#     return RandomEffectLogLikelihood(e_u, var_u, e_mu, var_mu, e_tau, e_log_tau)[0] + \
#            GammaEntropy(tau_shape, tau_rate) + \
#            e_log_p_tau[0]
#     return RandomEffectLogLikelihood(e_u, var_u, e_mu, var_mu, e_tau, e_log_tau)[0]
    return GammaEntropy(tau_shape, tau_rate)



class DebugWrapper():
    def __init__(self, glmm_par, prior_par, x_mat, y_vec, y_g_vec, num_draws):
        self.__glmm_par_ad = copy.deepcopy(glmm_par)
        self.__prior_par_ad = copy.deepcopy(prior_par)
        self.x_mat = x_mat
        self.y_vec = y_vec
        self.y_g_vec = y_g_vec
        draw_spacing = 1 / float(num_draws + 1)
        target_quantiles = np.linspace(draw_spacing, 1 - draw_spacing, num_draws)
        self.std_draws = sp.stats.norm.ppf(target_quantiles)

    def Eval(self, free_par_vec):
        self.__glmm_par_ad.set_free(free_par_vec)
        return DebugFun(self.__glmm_par_ad, self.__prior_par_ad)


debug_wrapper = DebugWrapper(glmm_par, prior_par, x_mat, y_vec, y_g_vec, 10)
DebugGrad = grad(debug_wrapper.Eval)
DebugHess = hessian(debug_wrapper.Eval)
KLHessVecProd = hessian_vector_product(debug_wrapper.Eval)  

free_par_vec = glmm_par_opt.get_free()
print debug_wrapper.Eval(free_par_vec)
foo = DebugHess(free_par_vec)
print foo.shape


prior_indices = copy.deepcopy(prior_par)
prior_indices.set_vector(1 + np.array(range(prior_indices.vector_size())))

vp_indices = copy.deepcopy(glmm_par_opt)
vp_indices.set_vector(1 + np.array(range(vp_indices.vector_size())))

inds = np.hstack([ vp_indices['tau'].shape.get() - 1, vp_indices['tau'].rate.get() - 1 ])
print foo[np.ix_(inds, inds)]

print kl_hess[np.ix_(inds, inds)]


-1.47116024586
(224, 224)
[[-0.00155647  0.        ]
 [ 0.         -0.00382605]]
[[ 52.49841289 -52.29835763]
 [-52.29835763  52.59365634]]
